- Just a training code/ Theoretical knowledge of Markov Models
- Training separate models for each class
- using Bayes Rule

In [1]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score

input_files = [
  'edgar_allan_poe.txt',
  'robert_frost.txt',
]

!head edgar_allan_poe.txt

--2023-03-07 16:57:21--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26622 (26K) [text/plain]
Saving to: ‘edgar_allan_poe.txt’

edgar_allan_poe.txt 100%[===================>]  26,00K  --.-KB/s    in 0,006s  

2023-03-07 16:57:21 (4,16 MB/s) - ‘edgar_allan_poe.txt’ saved [26622/26622]

--2023-03-07 16:57:21--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP re

In [2]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [3]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [4]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [5]:
len(Ytrain), len(Ytest)

(1615, 539)

In [6]:
train_text[:5]

['the truest the most fervently devoted',
 'and fight in a smother',
 'of what in other worlds shall be and given',
 'come up in despite of the lion',
 'sitting or standing as he chose']

In [7]:
Ytrain[:5]

[0, 1, 0, 0, 1]

In [8]:
idx = 1
word2idx = {'<unk>': 0}

In [9]:
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

In [10]:
word2idx

{'<unk>': 0,
 'the': 1,
 'truest': 2,
 'most': 3,
 'fervently': 4,
 'devoted': 5,
 'and': 6,
 'fight': 7,
 'in': 8,
 'a': 9,
 'smother': 10,
 'of': 11,
 'what': 12,
 'other': 13,
 'worlds': 14,
 'shall': 15,
 'be': 16,
 'given': 17,
 'come': 18,
 'up': 19,
 'despite': 20,
 'lion': 21,
 'sitting': 22,
 'or': 23,
 'standing': 24,
 'as': 25,
 'he': 26,
 'chose': 27,
 'all': 28,
 'we': 29,
 'can': 30,
 'hold': 31,
 'together': 32,
 'by': 33,
 'legs': 34,
 'thee': 35,
 'poetry': 36,
 'thy': 37,
 'presence': 38,
 'leaves': 39,
 'that': 40,
 'were': 41,
 'crisped': 42,
 'sere': 43,
 'town': 44,
 'is': 45,
 'no': 46,
 'more': 47,
 'from': 48,
 'sun': 49,
 'stars': 50,
 'whence': 51,
 'had': 52,
 'drawn': 53,
 'forth': 54,
 'those': 55,
 'name': 56,
 'stark': 57,
 'gathered': 58,
 'bow': 59,
 'i': 60,
 'paused': 61,
 'rested': 62,
 'on': 63,
 'sort': 64,
 'hook': 65,
 'could': 66,
 'see': 67,
 'nothing': 68,
 'toffile': 69,
 'dont': 70,
 'it': 71,
 'so': 72,
 'now': 73,
 'theyve': 74,
 'dragged

In [11]:
len(word2idx)

2514

In [12]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [13]:
train_text_int[100:105]

[[143, 386, 45, 109, 387, 6, 388],
 [71, 389, 81, 390],
 [26, 391, 357, 392, 6, 357, 393, 132, 394],
 [26, 395, 276, 396, 70, 40, 347, 124, 397],
 [326, 398, 351, 399, 400, 63, 109, 399]]

In [14]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [15]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [16]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [17]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [18]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

# P0 is about 33 percent and P one is about sixty six percent because of this, it wouldn't make sense to use the maximum likelihood method.
# Instead, we should look at the posterior, which corresponds to the map method.
# Note also that because the classes are slightly imbalanced, we may want to use a metric other than
# the accuracy to evaluate a classifier.





(0.34365325077399383, 0.6563467492260062)

In [20]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]
      
      # update last_idx
      last_idx = idx
    
    return logprob
  
  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [21]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [22]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.9962848297213622


In [23]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8460111317254174


In [24]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 549,    6],
       [   0, 1060]])

In [25]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[ 93,  70],
       [ 13, 363]])

In [26]:
f1_score(Ytrain, Ptrain)

0.9971777986829726

In [27]:
f1_score(Ytest, Ptest)

0.8974042027194068